https://machinelearningmastery.com/how-to-develop-lstm-models-for-time-series-forecasting/

<font size="5">**Data Preparation:**</font>

&emsp;&emsp;&emsp;&emsp;X (past ob.),&emsp; y (pred.)
<br>snapshot1: 10, 20, 30&emsp; &nbsp; 40
<br>snapshot2: 20, 30, 40&emsp; &nbsp; 50
<br>snapshot3: 30, 40, 50&emsp; &nbsp; 60

In [1]:
from LSTM import *
# define input sequence
raw_data = [10, 20, 30, 40, 50, 60, 70, 80, 90]
# choose a number of time steps
n_snapshots = 3
# split into samples
X, y = split_sequence(raw_data, n_snapshots)
# summarize the data
for i in range(len(X)):
    print(X[i], y[i])

[10 20 30] 40
[20 30 40] 50
[30 40 50] 60
[40 50 60] 70
[50 60 70] 80
[60 70 80] 90


<font size="5">**Vanilla LSTM**</font>

A Vanilla LSTM is an LSTM model that has a single hidden layer of LSTM units, and an output layer used to make a prediction.

In [2]:
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

2021-10-05 15:09:17.641790: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-10-05 15:09:17.641807: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
# reshape X from [samples, timesteps] into [samples, timesteps, features]
n_features = 1 # universal output
X = X.reshape((X.shape[0], X.shape[1], n_features))

# define model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_snapshots, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

2021-10-05 15:09:19.399925: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-10-05 15:09:19.400061: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2021-10-05 15:09:19.400146: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2021-10-05 15:09:19.405434: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory
2021-10-05 15:09:19.405491: W tensorflow/stream_executor/platform/default/dso_loader

In this case, we define a model with 50 LSTM units in the hidden layer and an output layer that predicts a single numerical value.

The model is fit using the [efficient Adam version of stochastic gradient descent](https://machinelearningmastery.com/adam-optimization-algorithm-for-deep-learning/) and optimized using the mean squared error, or *‘mse‘* loss function.

In [4]:
# fit model
model.fit(X, y, epochs=200, verbose=0)

2021-10-05 15:09:19.599789: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


We can predict the next value in the sequence by providing the input:

In [5]:
# demonstrate prediction
X_input = np.array([70, 80, 90])
X_input = X_input.reshape((1, n_snapshots, n_features))
print ('Input:\n', X_input)
y_pred = model.predict(X_input, verbose=0)
print ('Prediction:\n', y_pred)

Input:
 [[[70]
  [80]
  [90]]]
Prediction:
 [[101.76299]]


<font size="5">**Stacked LSTM**<font>

Multiple hidden LSTM layers can be stacked one on top of another in what is referred to as a Stacked LSTM model.

In [6]:
# define model
model = Sequential()
model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(n_snapshots, n_features)))
model.add(LSTM(50, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=200, verbose=0)
# demonstrate prediction
X_input = np.array([70, 80, 90])
X_input = X_input.reshape((1, n_snapshots, n_features))
print ('Input:\n', X_input)
y_pred = model.predict(X_input, verbose=0)
print ('Prediction:\n', y_pred)

Input:
 [[[70]
  [80]
  [90]]]
Prediction:
 [[104.27552]]


<font size="5">**Bidirectional LSTM**<font>

On some sequence prediction problems, it can be beneficial to allow the LSTM model to learn the input sequence both forward and backwards and concatenate both interpretations.

In [7]:
from keras.layers import Bidirectional

# define model
model = Sequential()
model.add(Bidirectional(LSTM(50, activation='relu'), input_shape=(n_snapshots, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=200, verbose=0)
# demonstrate prediction
X_input = np.array([70, 80, 90])
X_input = X_input.reshape((1, n_snapshots, n_features))
print ('Input:\n', X_input)
y_pred = model.predict(X_input, verbose=0)
print ('Prediction:\n', y_pred)

Input:
 [[[70]
  [80]
  [90]]]
Prediction:
 [[100.3004]]


<font size="5">**CNN LSTM**<font>

A convolutional neural network, or CNN for short, is a type of neural network developed for working with two-dimensional image data.

The CNN can be very effective at automatically extracting and learning features from one-dimensional sequence data such as univariate time series data.

A CNN model can be used in a hybrid model with an LSTM backend where the CNN is used to interpret subsequences of input that together are provided as a sequence to an LSTM model to interpret.

In [9]:
# define input sequence
raw_data = [10, 20, 30, 40, 50, 60, 70, 80, 90]
# choose a number of time steps
n_snapshots = 4
# split into samples
X, y = split_sequence(raw_data, n_snapshots)
# summarize the data
for i in range(len(X)):
    print(X[i], y[i])

[10 20 30 40] 50
[20 30 40 50] 60
[30 40 50 60] 70
[40 50 60 70] 80
[50 60 70 80] 90


Each sample can then be split into two sub-samples, each with two time steps. The CNN can interpret each subsequence of two time steps and provide a time series of interpretations of the subsequences to the LSTM model to process as input.

In [13]:
# reshape from [samples, timesteps] into [samples, subsequences, timesteps, features]
n_features = 1
n_seq = 2
n_steps = 2
X = X.reshape((X.shape[0], n_seq, n_steps, n_features))

In [19]:
from keras.layers import Flatten
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

# define model
model = Sequential()
model.add(TimeDistributed(Conv1D(filters=64, kernel_size=1, activation='relu'), input_shape=(None, n_steps, n_features)))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(50, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=500, verbose=0)
# demonstrate prediction
X_input = np.array([60, 70, 80, 90])
X_input = X_input.reshape((1, n_seq, n_steps, n_features))
print ('Input:\n', X_input)
y_pred = model.predict(X_input, verbose=0)
print ('Prediction:\n', y_pred)

Input:
 [[[[60]
   [70]]

  [[80]
   [90]]]]
Prediction:
 [[101.58238]]
